In [132]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from scipy.sparse import load_npz

In [119]:
def ProcessSingleTraj(Traj,TPMs,XInd,YInd):
    for cur in range(len(Traj)):
        for i,tpm in enumerate(TPMs):
            nex = i + 1 # indicates the future step 
            # If the Mea is not Identified
            if nex > (len(Traj) - 1):
                break 
            if (Traj[cur,0] != -1) & (Traj[nex,0] != -1):
                CurCellCode = CellCoding[XInd[cur],YInd[cur]]
                NexCellCode = CellCoding[XInd[nex],YInd[nex]]
                tpm[CurCellCode,NexCellCode] += 1


In [126]:
#read each traj and count freq
os.chdir(r'E:\Data\Verteran')
TrajPath = r'.\OutputFile\OutputTrajs'
TrajLis = os.listdir(TrajPath)
BinX,BinY = np.arange(-200,201),np.arange(-200,201)
CellCoding = np.arange(BinX.shape[0]*BinY.shape[0]).reshape((BinY.shape[0],BinX.shape[0])).astype(np.int32)
# Establish TPM for 0.7 second
TPMs = [lil_matrix((CellCoding.shape[0]*CellCoding.shape[1],CellCoding.shape[0]*CellCoding.shape[1]),dtype=np.int64) for i in range(7)]
for i in tqdm(range(len(TrajLis))):
    # rule out the estimated values by KF
    TestTraj = np.array(pd.read_csv(os.path.join(TrajPath,TrajLis[i])))[:,[0,1]][:-8]
    #XInd,YInd are digitized index from -200 to +200
    XInd,YInd = np.digitize(TestTraj[:,0],BinX),np.digitize(TestTraj[:,1],BinY)
    ProcessSingleTraj(TestTraj,TPMs,XInd,YInd)
# to calculate the normalized probability =
for tpm in tqdm(TPMs):
    for row in range(tpm.shape[0]):
        if tpm[row].sum() != 0:
            tpm[row] = tpm[row]/tpm[row].sum()

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [17:40<00:00, 151.45s/it]


In [133]:
csr_matrix(TPM[0])

<1x160801 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [135]:
for i,tpm in enumerate(TPMs):
    save_npz("E:/Data/Verteran/OutputFile/tpm_{}.npz".format(i), csr_matrix(tpm))
    # your_matrix_back = sparse.load_npz("yourmatrix.npz")

In [138]:
test = load_npz(r'E:/Data/Verteran/OutputFile/tpm_0.npz')

In [142]:
test

1841